In [1]:
import numpy as np
import pandas as pd
import os,sys

In [2]:
inputFiles = ['./HEPData-ins2080541-v1-csv/p0-valuesandmodel-independentlimits,longregime.csv',
              './HEPData-ins2080541-v1-csv/p0-valuesandmodel-independentlimits,shortregime.csv']

In [3]:
dataDict = {}

In [4]:
for inputFile in inputFiles:
    with open(inputFile, 'r') as f:
        data = f.read()

    blocks = data.split('#: Signal Region')
    blocks = blocks[1:]
    
    massWindowDict = {}
    for l in blocks[0].split('\n')[3:]:
        if not l.strip(): continue
        target,window = l.split(',')
        massWindowDict[eval(target)] = window    
                
    for b in blocks[1:]:
        if not b.replace(',','').strip():
            continue
        lines = b.split('\n')
        sr = lines[0].replace(',','').strip()
        regime = lines[1].split(',')[1]
        labels = lines[2].split(',')
        table = (sr,regime)
        if table not in dataDict:
            dataDict[table] = {'Target Mass [GeV]' : list(massWindowDict.keys()),
                               'Mass Window' : list(massWindowDict.values())}
        useDict = dataDict[table]    
        for label in labels:
            if not label in useDict:
                useDict[label] = np.zeros(len(massWindowDict))
        for l in lines[3:]:
            if not l.strip():
                continue
            vals = [eval(x) for x in l.split(',')]
            x = vals[0]
            ix = useDict['Target Mass [GeV]'].index(x)
            for il,label in enumerate(labels):
                useDict[label][ix] = vals[il]        

In [5]:
allData = None
for key,dd in dataDict.items():
    df = pd.DataFrame.from_dict(dd)
    df['SR'] = key[0]
    df['Regime'] = key[1]
    if allData is None:
        allData = df
    else:
        allData = pd.concat((allData,df))

In [6]:
allData

,Target Mass [GeV],Mass Window,Expected background,error +,error -,Observed,p0-value,Local Z signif.,S95_exp,S95_obs,SR,Regime
0,100,[120 200],68.0,7.0,-5.0,63.0,0.500000,0.0,19.0,16.0,SR-Inclusive_Low,Long Lifetime
1,200,[150 225],63.0,7.0,-4.0,54.0,0.500000,0.0,18.0,13.0,SR-Inclusive_Low,Long Lifetime
2,300,[250 350],40.9,6.0,-4.0,35.0,0.500000,0.0,15.0,11.0,SR-Inclusive_Low,Long Lifetime
3,400,[350 500],29.2,5.3,-3.2,33.0,0.254000,0.7,12.6,16.0,SR-Inclusive_Low,Long Lifetime
4,450,[400 550],21.5,4.2,-2.9,30.0,0.050300,1.6,11.0,19.4,SR-Inclusive_Low,Long Lifetime
...,...,...,...,...,...,...,...,...,...,...,...,...
12,1200,[800 2400],1.5,1.6,-0.8,6.0,0.009100,2.4,4.0,10.0,SR-Inclusive_High,Short Lifetime
13,1400,[900 2900],1.1,1.4,-0.7,7.0,0.002080,2.9,4.0,11.5,SR-Inclusive_High,Short Lifetime
14,1600,[1000 3450],0.9,1.5,-0.5,7.0,0.000603,3.2,3.6,11.8,SR-Inclusive_High,Short Lifetime
15,1800,[1100 4000],0.8,1.1,-0.2,7.0,0.000887,3.1,3.5,11.9,SR-Inclusive_High,Short Lifetime


In [7]:
allData.to_pickle('modelIndependentLimits.pcl')